In [1]:
import pandas as pd
import os
import sys
sys.path.append('../py')
from permute import permute_edges

In [2]:
edges = pd.read_csv('data/hetnet_baseedges.csv')

In [3]:
edges.head(2)

,:START_ID,:END_ID,:TYPE
0,Q221668,Q71969,drug-used-for-treatment_DduftC
1,Q246084,Q71969,drug-used-for-treatment_DduftC


In [4]:
n_perms = 5
multiplier = 5

In [5]:
%%time
stat_dfs = list()
permuted_edges = edges
 
for i in range(n_perms):
    i += 1
    print('Starting permutation', i)
    permuted_edges, stats = permute_edges(permuted_edges, multiplier=multiplier, seed=i)
    stat_df = pd.DataFrame(stats)
    stat_df['permutation'] = i
    stat_dfs.append(stat_df)
    path = os.path.join('../', 'neo', 'import', 'hetnet_baseedges_perm-{}.csv'.format(i))
    permuted_edges.to_csv(path, index = False)

# Save stats
stat_df = pd.concat(stat_dfs)
path = os.path.join('../', 'neo', 'import', 'stats.tsv')
stat_df.to_csv(path, sep='\t', index=False, float_format='%.5g')

Starting permutation 1
Starting permutation 2
Starting permutation 3
Starting permutation 4
Starting permutation 5
CPU times: user 13.7 s, sys: 0 ns, total: 13.7 s
Wall time: 13.7 s
